# Register the ML Model to an Azure ML Workspace

In this notebookw, we will register the trained machine learning model to an Azure Machine Learning Workspace. 

Azure ML provides a seamless way to register the model's artifacts to Azure ML for deployment and further management. This ensures reproducibility, versioning, and easy access to the model for future use.


## Configure AML client

In [1]:
import os
from dotenv import load_dotenv
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

load_dotenv()

subscription_id = os.environ.get("AZURE_SUBSCRIPTION_ID")
resource_group = os.environ.get("AZURE_RESOURCE_GROUP")
workspace = os.environ.get("WORKSPACE_NAME")

ml_client = MLClient(DefaultAzureCredential(), subscription_id, resource_group, workspace)

print("MLClient loaded successfully:", ml_client.workspace_name)

MLClient loaded successfully: wkmllab02


## Register the model to AML Workspace

In [3]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

file_model = Model(
    path="../train/model_artifacts/",
    type=AssetTypes.CUSTOM_MODEL,
    name="store-sales-forecasting",
    description="Model for store sales forecasting. Trained on the Store Sales - Time Series Forecasting dataset from Kaggle.",
    tags={"model_type": "regression", "dataset": "store-sales-forecasting"},
)

ml_client.models.create_or_update(file_model)

Uploading model_artifacts (52.49 MBs): 100%|██████████| 52493597/52493597 [00:04<00:00, 11686718.34it/s]




Model({'job_name': None, 'intellectual_property': None, 'system_metadata': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'store-sales-forecasting', 'description': 'Model for store sales forecasting. Trained on the Store Sales - Time Series Forecasting dataset from Kaggle.', 'tags': {'model_type': 'regression', 'dataset': 'store-sales-forecasting'}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/2168a4ef-0538-435e-9ae6-beb6183f2769/resourceGroups/RG-ML/providers/Microsoft.MachineLearningServices/workspaces/wkmllab02/models/store-sales-forecasting/versions/5', 'Resource__source_path': '', 'base_path': '/home/luiz-braz/workspace/reasoning-over-ml/train', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7fa81bfcf260>, 'serialize': <msrest.serialization.Serializer object at 0x7fa847faff80>, 'version': '5', 'latest_version': None, 'path': 'azureml://subscriptions/2168a4ef-0538-435e-9ae6-beb6183